In [53]:
pip install python-docx

In [54]:
from docx import Document
import tqdm
import os
import time
import string
import json
import re

Функция строки состояния, чтобы избежать бесконечных циклов

In [55]:
def spy(par_cur, par_len):
    done = "=" * (par_cur * 20 // par_len)
    undone = " " * (20 - (par_cur * 20 // par_len))
    percent = round(par_cur * 100 // par_len)
    print(f'[{done}{undone}]\t{percent}%', end='\r')

In [112]:
def getcitspan(text, addmeaning=False): # обработка спана примеров
    get_time = re.compile(r'(\d+.г\.|\d+–\d+.гг\.|[XVI]+–[XVI]+.вв\..?|[XVI]+.в\..\~.[XVI]+.в\..?|[XVI]+–[XVI]+.вв\..?|[XVI]+.в\..\~.[XVI]+.в\..?|[XVI]+.в\..\~.\d+.г\..?|[XVI]+.в\..?)')
    get_source = re.compile(r'(?<=[a-zа-яёα-ωѣ]\.) (?=[([{А-ЯЁѢ])')
    jfile = []
    runs = get_time.split(text) # деление текста примеров по значениям времени
    idx = 0
    while idx < len(runs)-1:
        try:
            if runs[idx][0].isalpha():
                inf = get_source.split(runs[idx]) # выделение источника
                if len(runs[idx]) > 0 and len(runs[idx+1]) > 0:
                    if len(inf) == 2 and (runs[idx+1][0].isnumeric() or runs[idx+1][0] in ('I', 'V', 'X')):
                        if '~' in runs[idx+1]:
                            try:
                                date, orig_date = runs[idx+1].split(' ~ ')
                                jfile.append({'text': inf[0], 'date': date, 'orig_date': orig_date, 'source': inf[1]})
                            except:
                                jfile.append({'text': inf[0], 'date': runs[idx+1], 'source': inf[1]})
                        else:
                            jfile.append({'text': inf[0], 'date': runs[idx+1], 'source': inf[1]})
                        if addmeaning != False:
                            jfile[-1]['side_meaning'] = addmeaning
                        idx += 2

                    elif len(inf) < 2 and (runs[idx+1][0].isnumeric() or runs[idx+1][0] in ('I', 'V', 'X')):
                        if '~' in runs[idx+1]:
                            try:
                                date, orig_date = runs[idx+1].split(' ~ ')
                                jfile.append({'text': runs[idx], 'date': date, 'orig_date': orig_date, 'source': ''})
                            except:
                                jfile.append({'text': runs[idx], 'date': runs[idx+1], 'source': ''})
                        else:
                            jfile.append({'text': runs[idx], 'date': runs[idx+1], 'source': ''})
                        if addmeaning != False:
                            jfile[-1]['side_meaning'] = addmeaning
                        idx += 2

                    else:
                        problem['c'].append(('inf is longer than 2', text, inf, runs[idx], runs[idx+1]))
                        idx += 1
                else:
                    problem['c'].append(('there are empty runs', text, runs[idx], runs[idx+1]))
                    idx += 1
            else:
                idx += 1
        except:
            idx += 1
    return jfile

In [58]:
def getcitlist(text): # обработка примеров
    jfile = []
    if len(text.split(' ||\xa0')) == 1: # если в примерах не указаны подзначения
        jfile.extend(getcitspan(text))

    else: # случай, если есть подзначения
        first = text.split(' ||\xa0')[0]
        jfile.extend(getcitspan(first))

        for citation in text.split(' ||\xa0')[1:]: # примеры с подзначением получают дополнительное поле
            side_meaning = citation.split('. ')[0].strip() + '.'
            cit = citation.replace(side_meaning, '').strip()
            jfile.extend(getcitspan(cit, side_meaning))
    return jfile

In [59]:
def getnumber(non_empty): # выделение номера значения
    run_cur = 0

    if non_empty[run_cur].text.strip(string.punctuation+' ').isnumeric():
        number = non_empty[run_cur].text.strip(string.punctuation+' ')
        run_cur += 1
        while non_empty[run_cur].text.strip(string.punctuation+' ').isnumeric(): # случай выделения числа в разные спаны
            number += non_empty[run_cur].text.strip(string.punctuation+' ')
            run_cur += 1
    elif len(non_empty[run_cur].text.split()) > 0 and \
                                        non_empty[run_cur].text.split()[0].strip(string.punctuation+' ').isnumeric():
         # случай невыделения числа в отдельный спан
        number = non_empty[run_cur].text.split()[0].strip(string.punctuation+' ')
    elif non_empty[run_cur].text.strip(string.punctuation+' ') in ('', '◊', '–'): # случай устойчивых выражений
        number = '◊'
        run_cur += 1
    elif non_empty[run_cur].text.strip(string.punctuation+' ') in ('I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X'):
         # случай омонимичных элементов
        number = non_empty[run_cur].text.strip(string.punctuation+' ')
        run_cur += 1
    else: # случай единственного значения
        number = '0'

    return number, non_empty[run_cur:]

In [95]:
def getdef(runs, non_empty): # выделение значения
    gram = {'c', 'с', 'глаг', 'ж', 'м',
            'мест', 'мн', 'нареч', 'предлог', 'прил',
            'союз', 'частица', 'собир', 'ж., собир',
            'м. собир', 'с. собир', 'прил. неизм', 'неизм',
            'ж. неизм', 'м. неизм', 'с. неизм'}

    getref = re.compile(r'То же, что( \w+){1,2} \(.+?\)\.')
    getref2 = re.compile(r'То же, что( \w+){1,2} \(.+?\)\;')
    getref3 = re.compile(r'Уменьш.-ласк. к( \w+){1,2} \(.+\)\.')
    getref4 = re.compile(r'см\. (\w+){1,2} в знач\. \d+')
    getref5 = re.compile(r'см\. (\w+){1,2} \(в знач\. \d+\)')
    getref6 = re.compile(r'см\. (\w+){1,2}')


    reference = getref.search(''.join([run.text for run in runs]))
    if reference != None:
        txt = reference.group()
        run_cur = 0
        while run_cur < len(non_empty) and non_empty[run_cur].text not in txt:
            run_cur += 1
        while run_cur < len(non_empty) and non_empty[run_cur].text in txt:
            run_cur += 1
        return txt, non_empty[run_cur:]

    reference = getref2.search(''.join([run.text for run in runs]))
    if reference != None:
        txt = reference.group()
        run_cur = 0
        while run_cur < len(non_empty) and non_empty[run_cur].text not in txt:
            run_cur += 1
        while run_cur < len(non_empty) and non_empty[run_cur].text in txt:
            run_cur += 1
        return txt, non_empty[run_cur:]

    reference = getref3.search(''.join([run.text for run in runs]))
    if reference != None:
        txt = reference.group()
        run_cur = 0
        while run_cur < len(non_empty) and non_empty[run_cur].text not in txt:
            run_cur += 1
        while run_cur < len(non_empty) and non_empty[run_cur].text in txt:
            run_cur += 1
        return txt, non_empty[run_cur:]

    reference = getref4.search(''.join([run.text for run in runs]))
    if reference != None:
        txt = reference.group()
        run_cur = 0
        while run_cur < len(non_empty) and non_empty[run_cur].text not in txt:
            run_cur += 1
        while run_cur < len(non_empty) and non_empty[run_cur].text in txt:
            run_cur += 1
        return txt, non_empty[run_cur:]

    reference = getref5.search(''.join([run.text for run in runs]))
    if reference != None:
        txt = reference.group()
        run_cur = 0
        while run_cur < len(non_empty) and non_empty[run_cur].text not in txt:
            run_cur += 1
        while run_cur < len(non_empty) and non_empty[run_cur].text in txt:
            run_cur += 1
        return txt, non_empty[run_cur:]

    reference = getref6.search(''.join([run.text for run in runs]))
    if reference != None:
        txt = reference.group()
        run_cur = 0
        while run_cur < len(non_empty) and non_empty[run_cur].text not in txt:
            run_cur += 1
        while run_cur < len(non_empty) and non_empty[run_cur].text in txt:
            run_cur += 1
        return txt, non_empty[run_cur:]

    run_cur = -1
    idx_start = -1

    if run_cur == -1:
        run_cur = 0
    if idx_start == -1:
        idx_start = runs.index(non_empty[run_cur])

    while run_cur < len(non_empty) and (non_empty[run_cur].italic or \
                        (non_empty[run_cur].text.strip() and non_empty[run_cur].text.strip()[0].islower())):
        run_cur += 1

    try:
        idx_end = runs.index(non_empty[run_cur])
        txt = ''.join([run.text for run in runs[idx_start:idx_end]]) # восстановение вида значения с пунктуацией
    except:
        idx_end = runs.index(non_empty[run_cur-1])
        txt = ''.join([run.text for run in runs[idx_start:idx_end]]) # восстановение вида значения с пунктуацией
    try:
        if idx_end < len(runs) and (non_empty[run_cur].text.split() != [] or runs[idx_end].text.split() != []):
            # добавление конечной пунктуации
            if runs[idx_end].text.split() == []:
                if non_empty[run_cur].text.split()[0].strip(string.punctuation) == '':
                    txt += non_empty[run_cur].text.split()[0]
            elif runs[idx_end].text.split()[0].strip(string.punctuation) == '':
                txt += runs[idx_end].text.split()[0]
        return txt, non_empty[run_cur:]
    except:
        return txt, non_empty
        problem['d'].append(runs)

In [105]:
def parsesense(runs, gram=''): # обработка значения
    jfile = {}
    non_empty = []

    i = 0
    while i < len(runs): # чистка вхождений от пустых спанов
        if runs[i].text.strip(string.punctuation+' ') != '' or runs[i].text == '?':
            non_empty.append(runs[i])
        i += 1

    if len(non_empty) == 0:
        return

    jfile['num'], non_empty = getnumber(non_empty) # выделение номера

    if len(non_empty) == 0:
        return

    run_cur = 0
    additional = ''
    while run_cur < len(non_empty) and not non_empty[run_cur].italic: # выделение дополнительной информации
        additional += non_empty[run_cur].text
        run_cur += 1
    jfile['additional'] = additional

    if run_cur >= len(non_empty):
        problem['s'].append(runs)
        return

    definition, non_empty = getdef(runs, non_empty[run_cur:]) # выделение значения

    if gram != '':
         # удаление грамматической информации из значения
        definition = re.sub(fr'\A\s*{gram}\.?', '', definition)
    if not len(definition.strip()) > 0:
        problem['s'].append(runs)
    elif definition.strip()[0].isnumeric():
         # удаление номера из значения
        definition = re.sub(fr'\A\s*\d+\.?\s*', '', definition)

    jfile['definition'] = definition

    if len(non_empty) == 0:
        return

    idx = runs.index(non_empty[0]) # удаление лишних символов из значений
    rest = ''.join([run.text for run in runs[idx:]])
    rest = rest[1:] if rest[0] == '.' else rest
    rest = rest[1:] if rest[0] == ' ' else rest

    jfile['citations'] = getcitlist(rest) # выделение примеров
    return jfile

In [62]:
def getgram(runs): # выделение грамматической информации
    gram = {'c', 'с', 'глаг', 'ж', 'м',
            'мест', 'мн', 'нареч', 'предлог', 'прил',
            'союз', 'частица', 'собир', 'ж., собир',
            'м. собир', 'с. собир', 'прил. неизм', 'неизм',
            'ж. неизм', 'м. неизм', 'с. неизм'}
    gram_in_sense = ''
    run_cur = 0
    grammar = ''

    if runs[run_cur].text.strip(' .,') in gram:
        grammar = runs[run_cur].text.strip(' .,')
        run_cur += 1
        if runs[run_cur].text.strip()  in ('.', '.,', ',', ''):
            run_cur += 1
        if runs[run_cur].text.strip(' .') in gram: # выделение второго грам. показателя
            grammar = grammar + '., ' + runs[run_cur].text.strip(' .,')
            run_cur += 1
        elif runs[run_cur].text.split() != [] and \
                                            runs[run_cur].text.split()[0].strip('.,') in gram:
            # выделение второго грам. показателя в случае его невыделения в отдельный спан
            grammar = grammar + '., ' + runs[run_cur].text.split()[0].strip('.,')
            gram_in_sense = runs[run_cur].text.split()[0].strip('.,')

    elif runs[run_cur].text != '\xa0' and runs[run_cur].text.split()[0].strip('.,') in gram:
        # выделение грамматического показателя в случае его невыделения в отдельный спан
        if runs[run_cur].text.split()[1].strip('.,') in gram:
            grammar = ' '.join(runs[run_cur].text.split()[:2]).strip('.,')
        else:
            grammar = runs[run_cur].text.split()[0].strip('.,')
        gram_in_sense = grammar

    return grammar, gram_in_sense, runs[run_cur:]

In [63]:
def getetym(runs):
    etym = re.compile(r'\A\(ср\. .+?\).')
    text = ''.join([run.text for run in runs])
    text = text.strip(' .,')

    find = etym.search(text)
    if find != None:
        etymology = find.group()
        run_cur = 0
        while run_cur < len(runs) and runs[run_cur].text.strip(' .,') in etymology:
            run_cur += 1
        return etymology, runs[run_cur:]
    return None, runs

In [64]:
def parseentry(paragraphs): # обработка входа
    jfile = {}
    for paragraph in paragraphs:
        if paragraph.text.strip(' \n'+string.punctuation) == '':
            continue # пустые абзацы пропускаем и переходим к следующим абзацам

        # если первый ран абзаца болдовый или если первый ран - скобка, а второй болдовый, то это начало статьи
        if paragraph.runs[0].bold or \
                        (len(paragraph.runs)>1 and paragraph.runs[1].bold and paragraph.runs[0].text in ('[', '(', '<', '{')):
            run_cur = 0

            while run_cur<len(paragraph.runs) and (paragraph.runs[run_cur].bold \
                                or paragraph.runs[run_cur].text.strip() in ('и', 'или', ',') \
                                or paragraph.runs[run_cur].text.strip(string.punctuation+ ' ') == ''): # выделение форм
                run_cur += 1

            if run_cur>=len(paragraph.runs):
                problem['e'].append(paragraph.text)
                return

            forms_txt = ''.join([run.text for run in paragraph.runs[:run_cur]])
            splitter = re.compile(r'\s?(,|или|и| )\s?')
            jfile['form'] = ''
            jfile['addforms'] = []
            for form in splitter.split(forms_txt): # классификация форм
                if form.strip('()[]<>{}').isupper():
                    if jfile['form'] == '':
                        jfile['form'] = form
                    else:
                        jfile['addforms'].append(form)

            while paragraph.runs[run_cur].text.strip(string.punctuation+' ') == '':
                run_cur += 1

            if paragraph.runs[run_cur].text.strip(string.punctuation+' ') == 'см': # случай ссылки
                jfile['reference'] = ''.join([run.text for run in paragraph.runs[run_cur:]])
                continue
            if paragraph.runs[run_cur].text.strip(string.punctuation+' ') in ('и произ', 'и произв',
                                                                            'и произв. см', 'и произ. см'): # случай ссылки 2
                jfile['reference'] = ''.join([run.text for run in paragraph.runs[run_cur:]])
                continue

            while paragraph.runs[run_cur].text.strip(string.punctuation+' ') == '':
                run_cur += 1

            jfile['grammar'], gram_in_sense, runs = getgram(paragraph.runs[run_cur:]) # выделение грамматической информации
            run_cur = 0

            while runs[run_cur].text.strip() in ('', ',', 'и', 'или', '.', '. и', '. или'):
                run_cur += 1

            if runs[run_cur].bold and runs[run_cur].text.strip(string.punctuation+' ').isupper():
                start = run_cur
                while runs[run_cur].bold or runs[run_cur].text.strip() in ('и', 'или', ',') \
                                        or runs[run_cur].text.strip(string.punctuation+ ' ') == '': # выделение форм
                    run_cur += 1
                forms_txt = ''.join([run.text for run in runs[start:run_cur]])
                form_list = []
                for form in splitter.split(forms_txt): # классификация форм
                    if form.strip('()[]<>{}').isupper():
                        form_list.append(form)
                grammar, gram_in_sense, runs = getgram(runs[run_cur:])
                jfile['addforms'].append({'forms': form_list, 'grammar': grammar})

            etym, runs = getetym(runs)
            if etym != None:
                jfile['etymology'] = etym

            meaning = parsesense(runs, gram=gram_in_sense)
            if meaning != None:
                jfile['meanings'] = [meaning] # выделенние значения

        # а если абзац начинаестя с рана с цифрой или вот чем-то таким, то это абзац с дополнительным значением
        elif len(paragraph.text.strip())>0 and paragraph.text.strip()[0].isnumeric() or\
                            paragraph.text.strip()[0] in ('', '◊', '–') or\
                            paragraph.text.strip()[0] in ('I', 'V', 'X') or paragraph.text[0] in ('', ' ', '\xa0'):
            # определение абзаца со значением
            meaning = parsesense(paragraph.runs)
            if meaning != None:
                try:
                    jfile['meanings'].append(meaning) # выделение значения
                except:
                    jfile['meanings'] = [meaning]

    return jfile

In [103]:
def findentries(paragraphs): # выделение абзацев, входящих в одну словарную статью
    entries = []
    par_len = len(paragraphs)
    par_cur = 0
    start = 0
    while par_cur < par_len - 1:
        spy(par_cur, par_len)
        paragraph = paragraphs[par_cur]

        run_cur = 0
        while run_cur < len(paragraph.runs)-1 and paragraph.runs[run_cur].text.strip(string.punctuation+' ') == '':
            run_cur += 1

        if len(paragraph.runs) > 0 and paragraph.runs[run_cur].bold: # первый абзац статьи
            start = par_cur

        if not paragraphs[par_cur + 1].runs:# or paragraphs[par_cur + 1].runs[0].bold:
            entries.append((start, par_cur))

        if len(paragraphs[par_cur + 1].runs) > 0 and paragraphs[par_cur + 1].runs[0].bold:
            entries.append((start, par_cur))

        par_cur += 1

    paragraph = paragraphs[par_cur] # смотрим на последний абзац (?)

    run_cur = 0
    while run_cur < len(paragraph.runs)-1 and paragraph.runs[run_cur].text.strip(string.punctuation+' ') == '':
        run_cur += 1

    try:
        if paragraph.runs[run_cur].bold:
            entries.append((par_cur, par_cur))
        else:
            entries.append((start, par_cur))
    except:
        pass

    return entries

In [66]:
def parsedct(doc):
    jfile = []
    for start, end in tqdm.tqdm(findentries(doc.paragraphs)):
        jfile.append(parseentry(doc.paragraphs[start: end+1]))
    return jfile

In [107]:
docs2tei = ['28.docx', '29.docx', '30.docx', '31.docx']

In [113]:
for d in docs2tei:
    name = d
    d = Document(d)
    problem = {'s': [], 'e': [], 'd': [], 'c':[]}
    parsed = parsedct(d)
    print(name, len(problem['d']), len(problem['e']), len(problem['s']), len(problem['c']))
    with open(f'{name[0:2]}.json', 'w', encoding='utf-8') as f:
        json.dump(parsed, f, ensure_ascii=False, indent='\t')

100%|██████████| 1725/1725 [00:41<00:00, 41.25it/s]


28.docx 0 1 13 2230


100%|██████████| 2508/2508 [02:34<00:00, 16.26it/s]


29.docx 0 3 5 3293


100%|██████████| 2056/2056 [00:41<00:00, 49.83it/s]


30.docx 0 5 3 2896


100%|██████████| 1794/1794 [00:35<00:00, 50.39it/s]


31.docx 0 34 94 2479


# Раздел значений

In [ ]:
with open('sense_errs.txt', 'w', encoding='utf-8') as f:
    for idx, pr in enumerate(problem['s']):
        problematic_text = ''.join([run.text for run in pr])
        f.write(f'{idx}.\n{problematic_text}\n―――――――――――――――――――――――――――――――――――――――――――――――――\n')

In [ ]:
with open('cit_errs.txt', 'w', encoding='utf-8') as f:
    for idx, pr in enumerate(problem['c']):
        problematic_text = pr[0] + '\nin\n' + pr[0]
        f.write(f'{idx}.\n{problematic_text}\n―――――――――――――――――――――――――――――――――――――――――――――――――\n')